In [1]:
# Install dependencies
%pip install -q amplpy

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from amplpy import AMPL, ampl_notebook
from google.colab import userdata # for accessing colab secrets

ampl = ampl_notebook(
    modules=["cbc"],
    license_uuid=userdata.get('license'),  # license from "secret"
)   # instantiate AMPL object and register magics

ModuleNotFoundError: No module named 'google'

In [ ]:
%%writefile product_mix.mod
# file: product_mix.mod
#
# Product mix problem
#
# Maximize profit subject to less than or equal to constraints

# Sets
set Products;
set Resources;

# Decision variables
var x {j in Products} >= 0;

# Parameters
param profit {j in Products} >= 0;
param availability {i in Resources} >= 0;
param consumption {i in Resources, j in Products} >= 0;

# Objective function
maximize total_profit: sum {j in Products} profit[j]*x[j];

# Constraints
s.t.   usage {i in Resources}:
	sum {j in Products} consumption[i, j]*x[j]
		<=  availability[i];


In [ ]:
%%writefile product_mix.dat
# file: product_mix.dat

set Products := prod_1 prod_2;
set Resources := raw_material labour demand;
param profit :=
    prod_1 120
    prod_2 40;
param availability :=
	raw_material 2200
	labour        320
	demand        100;
param consumption: prod_1 prod_2 :=
	raw_material 40 20
	labour        8  2
	demand        1  1;


In [ ]:
%%ampl_eval
# file: product_mix.run

model product_mix.mod;
data product_mix.dat;
show;
expand;
option solver cbc;
solve;
display total_profit;
display x;
display usage;
